In [1]:
# Importamos módulos necesarios
import os  # Para acceder a variables de entorno
from dotenv import load_dotenv  # Para cargar variables de entorno desde un archivo .env
from openai import OpenAI  # Cliente para acceder a la API de OpenAI
import requests  # Para hacer solicitudes HTTP (a APIs)
import json  # Para manejar datos en formato JSON

# Carga las variables de entorno definidas en un archivo .env
load_dotenv()

# Inicializa el cliente de OpenAI usando la clave API desde las variables de entorno
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Función para obtener el clima actual de una ubicación usando la API de Open-Meteo
def get_weather(latitude: float, longitude: float) -> str:
    print("Getting weather...")  # Mensaje para indicar que se está obteniendo el clima
    # URL de la API con la latitud y longitud proporcionadas
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)  # Hacemos la petición a la API
    weather_data = response.json()  # Convertimos la respuesta a JSON

    return json.dumps(weather_data)  # Devolvemos los datos del clima como string JSON

# Lista de mensajes que simulan una conversación con el asistente
messages = [
    {
        "role": "system",
        "content": "Eres un asistente que entrega datos sobre el clima del mundo en tiempo real usando la función get_weather"
    },
    {
        "role": "user",
        "content": "¿Cuál es el clima de madrid?"  # Pregunta del usuario
    }
]

# Lista de funciones que el modelo puede llamar
functions = [
    {
        "type": "function",  # Tipo de herramienta: función
        "function": {
            "name": "get_weather",  # Nombre de la función que el modelo puede usar
            "description": "Usa esta funcion para obtener información sobre el clima",
            "parameters": {  # Parámetros que acepta la función
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Latitud de la ubicación"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Longitud de la ubicación"
                    }
                },
                "required": ["latitude", "longitude"]  # Parámetros obligatorios
            },
            "output": {
                "type": "string",
                "description": "Clima de la ubicación pedida por el usuario"
            }
        }
    }
]

# Se envía la conversación y funciones disponibles al modelo GPT-4o
response = client.chat.completions.create(
    model="gpt-4o",  # Modelo de OpenAI usado
    messages=messages,  # Mensajes del chat
    tools=functions  # Funciones que puede usar el modelo
)

# Guardamos el mensaje de respuesta del asistente
assistant_message = response.choices[0].message

print("Respuesta del asistente")
print(assistant_message)

# Si el asistente decide llamar una función:
if assistant_message.tool_calls:
    for tool_call in assistant_message.tool_calls:
        if tool_call.type == "function":
            function_name = tool_call.function.name
            # Convertimos los argumentos de la función de string JSON a diccionario
            function_args = json.loads(tool_call.function.arguments)

            # Si el asistente eligió la función "get_weather"
            if function_name == "get_weather":
                print(f"El asistente está llamando a la función get_weather")
                # Llamamos la función local con los argumentos proporcionados por el modelo
                weather_info = get_weather(
                    latitude=function_args.get("latitude"),
                    longitude=function_args.get("longitude")
                )

                # Añadimos el mensaje del asistente y la respuesta de la función a la conversación
                messages.append(assistant_message)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": weather_info
                })

# Hacemos otra llamada al modelo, ahora con la conversación extendida que incluye la respuesta del clima
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

# Obtenemos la respuesta final del asistente después de obtener el clima
final_reply = second_response.choices[0].message.content

print("Respuesta final del asistente")
print(final_reply)


Respuesta del asistente
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qLp1pGYpl8PaZrGaoN99PMo3', function=Function(arguments='{"latitude":40.4168,"longitude":-3.7038}', name='get_weather'), type='function')], annotations=[])
El asistente está llamando a la función get_weather
Getting weather...
Respuesta final del asistente
El clima actual en Madrid es el siguiente:

- Temperatura: 18.1°C
- Velocidad del viento: 13.8 km/h
- Dirección del viento: 49°
- Es de día.

El código meteorológico indica un clima despejado.
